In [ ]:
from IPython.core.display import HTML
HTML("<style>.container { width:95% !important; }</style>")

# Lecture 7, direct methods for constrained optimization

Direct methods for constrained optimization are also known as *methods of feasible directions*

## Feasible descent directions

Let $S\subset \mathbb R^n$ ($S\neq \emptyset$ closed) and $x^*\in S$. 
**Definition:** The set
$$ D = \{d\in \mathbb R^n: d\neq0,x^*+\alpha d\in S \text{ for all } \alpha\in (0,\delta) \text{ for some } \delta>0\}$$
is called the cone of feasible directions of $S$ in $x^*$.

**Definition:** The set 
$$ F = \{d\in \mathbb R^n: f(x^*+\alpha d)<f(x^*)\text{ for all } \alpha\in (0,\delta) \text{ for some } \delta>0\}$$
is called the cone of descent directions.

**Definition:** The set $F\cap D$ is called the cone of feasible descent directions.

![alt text](images/feasible_descent_directions.svg "Feasible descent directions")

**(Obvious) Theorem:** Consider an optimization problem 
$$
\begin{align}
\min &\  f(x)\\
\text{s.t. }&\ x\in S
\end{align}
$$
and let $x^*\in S$. Now if $x^*$ is a local minimizer **then** the set of feasible descent directions $F\cap D$ is empty.

## Idea for the methods of feasible descent directions

1. Assume a feasible solution $x$.
2. Find a feasible descent direction $d\in D\cap F$.
3. Determine the step length to the direction $d$
4. Update $x$ accordingly.




# Rosen's projected gradient method

Assume a problem with linear equality constraints

$$
\min f(x)\\
\text{s.t. }Ax=b,
$$

where $A$ is a $l\times n$ matrix ($l\leq n$) and $b$ is a vector.

Let $x$ be a feasible solution to the above problem.

It holds that:

$d$ is a feasible direction *if and only if* $Ad=0$

Thus, the gradient $-\nabla f(x)$ is a feasible descent direction, if 

$$ A\nabla f(x)=0.$$

This may or may not be true (i.e. the gradient may or may not be a feasible direction).

However, we can project the gradient to the set of feasible descent directions
$$ \{d\in \mathbb R^n: Ad=0\},$$
which now is a linear subspace.

![alt text](images/subspace.svg "A linear subspace Ad=0")

### Projection

Let $a\in \mathbb R^n$ be a vector and let $L$ be a linear subspace of $\mathbb R^n$. Now, the following are equivalent
* $a^P$ is the projection of $a$ on $L$,
* $\{a^P\} = \operatorname{argmin}_{l\in L}\|a-l\|$, and
* $a^P\in A$ and $(a-a^P)^Tl=0$ for all $l\in L$.

## Projected gradient

The projection of the gradient $\nabla f(x)$ on the set $\{d\in \mathbb R^n: Ad=0\}$ is denoted by $\nabla f(x)^P$ and called the *projected gradient*. 

Now, given some conditions, the projected gradient gives us a feasible descent direction.

![alt text](images/projected_gradient.svg "A projected gradient")

## How to compute the projected gradient?

There are different ways, but at this course we can use optimization. Basically, the optimization problem that we have to solve is
$$
\min \|\nabla f(x)-d\|\\
\text{s.t. }Ad=0.
$$

Since it is equivalent to minimize the square of the objective function $\sum_{i=n}\nabla_i f(x)^2+d_i^2-2\nabla_i f(x)d_i$, we can see that the problem is a quadratic problem with equality constraints,
$$
\min \frac12 d^TId-\nabla f(x)^Td\\
\text{s.t. }Ad=0
$$
which means that we just need to solve the system of equations (see e.g., https://en.wikipedia.org/wiki/Quadratic_programming#Equality_constraints)

$$
\left[
\begin{array}{cc}
I&A^T\\
A&0
\end{array}
\right] 
\left[\begin{align}d\\\lambda\end{align}\right]
= \left[ 
\begin{array}{c}
\nabla f(x)\\
0
\end{array}
\right],
$$

where I is the identity matrix, and $\lambda$ are the KKT multipliers.

### Code in Python

#### A function for projecting a vector to a linear space defined by $Ax=0$.

In [ ]:
import numpy as np
help(np.linalg.solve)

In [ ]:
import numpy as np
def project_vector(A,vector):
    #convert A into a matrix
    A_matrix = np.matrix(A)
    #construct the "first row" of the matrix [[I,A^T],[A,0]]
    left_matrix_first_row = np.concatenate((np.identity(len(vector)),A_matrix.transpose()), axis=1)
    #construct the "second row" of the matrix
    left_matrix_second_row = np.concatenate((A_matrix,np.matrix(np.zeros([len(A),len(A)]))), axis=1)
    #combine the whole matrix by combining the rows
    left_matrix = np.concatenate((left_matrix_first_row,left_matrix_second_row),axis = 0)
    #Solve the system of linear equalities from the previous page
    return np.linalg.solve(left_matrix, \
                           np.concatenate((np.matrix(vector).transpose(),\
                                           np.zeros([len(A),1])),axis=0))[:len(vector)] 

In [ ]:
# Example: Project gradient such that A*proj_gradient = 0
A = [[1,0,0],[0,1,0]]
gradient = [1,1,1]
project_vector(A,gradient)
#print len(A),len(gradient),len(A[0])

# Example

Let us study optimization problem
$$
\begin{align}
\min \qquad& x_1^2+x_2^2+x_3^2\\
\text{s.t.}\qquad &x_1+x_2=3\\
    &x_1+x_3=4.
\end{align}
$$

Let us project a negative gradient from a feasible point $x=(1,2,3)$

Now, the matrix
$$
A = \left[
\begin{array}{ccc}
1& 1 & 0\\
1& 0 & 1
\end{array}
\right]
$$.

In [ ]:
import ad
A = [[1,1,0],[1,0,1]]
gradient = ad.gh(lambda x:x[0]**2+x[1]**2+x[2]**2)[0]([1,2,3])
print(gradient)
d = project_vector(A,[-i for i in gradient])
print(d)

### d is a feasible direction

In [ ]:
np.matrix(A)*d

### d is a descent direction

In [ ]:
def f(x):
    return x[0]**2+x[1]**2+x[2]**2
alpha = 0.001
print("Value of f at [1,2,3] is "+str(f([1,2,3])))
x_mod= np.array([1,2,3])+alpha*np.array(d).transpose()[0]
print(x_mod)
print("Value of f at [1,2,3] +alpha*d is "+str(f(x_mod)))
print("Gradient dot product direction (i.e., directional derivative) is " \
      + str(np.matrix(ad.gh(f)[0]([1,2,3])).dot(np.array(d))))

## Finally, the algorithm of the projected gradient

In [ ]:
import numpy as np
import ad
def projected_gradient_method(f,A,start,step,precision):
    f_old = float('Inf')
    x = np.array(start)
    steps = []
    f_new = f(x)
    iters = 0
    # this will be filled in during the lecture
    while abs(f_old-f_new)>precision:
        # store the current function value
        f_old = f_new
        # compute gradient
        gradient = ad.gh(f)[0](x)
        # project negative gradient
        d = project_vector(A,[-i for i in gradient])
        # take transpose
        d = d.reshape(1,-1)
        # take step
        x = np.array(x + step*d)[0]
        # compute f in new point
        f_new = f(x)
        # record new step
        steps.append(x)
        # update iterations counter
        iters = iters + 1
    return x,f_new,steps,iters

In [ ]:
f = lambda x:x[0]**2+x[1]**2+x[2]**2
A = [[1,1,0],[1,0,1]]
start = [1,2,3]
(x,f_val,steps,iters) = projected_gradient_method(f,A,start,0.6,0.000001)

In [ ]:
print(x)
print(f(x))
print(f([1,2,3]))
print(np.matrix(A)*np.matrix(x).transpose())
print(iters)

## Note
If there are both linear equality and inequality constraints, the projection matrix does not remain the same 
* At each iteration, it includes only the equality and active inequality constraints

# Active set method
Consider a problem
$$
\min f(x)\\
\text{s.t. }Ax\leq b,
$$
where $A$ is a $l\times n$ matrix ($l\leq n$) and $b$ is a vector.

## Idea
* In $𝑥^ℎ$, the set of constraints is divided into active ($𝑖 ∈ 𝐼$) and inactive constraints
* Inactive constraints are not taken into account when the search direction $𝑑^ℎ$ is determined
* Inactive constraints affect only when computing the optimal step length $\alpha^ℎ$


## Feasible directions
* For $𝑖\in 𝐼$ , $(𝑎^i)^Tx^h = b_i$
* If $𝑑^ℎ$ is feasible in $𝑥^ℎ$, then $𝑥^ℎ + \alpha 𝑑^ℎ \in 𝑆$ for some $\alpha > 0$
* $(𝑎^i)^T(x^h+\alpha d^h) = (a^i)^Tx^h + \alpha(a^i)^Td^h\leq b_i$
* $(𝑎^i)^Td^h\leq 0$ for feasible $𝑑^ℎ$ and the constraint remains active if $(𝑎^i)^Td^h=0$

## On active constraints
* Optimization problem with inequality constraints is more difficult than problem with equality constraints since the active set in a local minimizer is not known
* If it would be known, then it would be enough to solve a corresponding equality constrained problem
* In that case, if the other constraints would be satisfied in the solution and all the Lagrange multipliers were non-negative, then the solution would also be a solution to the original problem

## Using active set
* At each iteration, a working set is considered which consists of the active constraints in $𝑥^ℎ$
* The direction $𝑑^ℎ$ is determined so that it is a descent direction in the working set
  * E.g. Rosen’s projected gradient method can be used

## Active set algorithm
1. Choose a starting point $x^1$ and determine an initial active set $I^1$ and set $h=1$
2. Compute a feasible descent direction $d^h$ in the subspace defined by the active constraints (e.g. by using projected gradient)
3. If $||d^h||=0$, go to step 6, otherwise, find optimal step length $\alpha$ by staying in the feasible set and set $x^{h+1} = x^h + \alpha d^h$
4. If no new constraint becomes active go to step 7 (active set does not change)
5. Addition to active set: a new constraint $j$ becomes active, update $I^{h+1} = I^h \cup \{j\}$ and go to step 7
6. Removal from active set: approximate Lagrangian multipliers $\mu_i$, $i\in I^h$. If $\mu_i\geq 0$ for all $i$, stop (active set is correct). Otherwise, remove a constraint $j$ with negative multiplier from the active set: $I^{h+1}=I^h\setminus \{j\}$
7. Set $h=h+1$ and go to step 2

<i>Implementation of the active set method is left as a voluntary exercise </i>

More information about alternative way of computing the projected gradient and approximation for the Lagrange multipliers can be found in
* <a href="http://users.jyu.fi/~jhaka/opt/TIES483_constrained_direct1.pdf">Constrained optimization: direct methods (Part 1)</a>
* <a href="http://users.jyu.fi/~jhaka/opt/TIES483_constrained_direct2.pdf">Constrained optimization: direct methods (Part 2)</a>